In [46]:
from financial_statements_extractor import financial_statements_extractor

extractor = financial_statements_extractor()

ImportError: attempted relative import with no known parent package

In [43]:
# 00266961
dart.list('00164742', start='1999-01-01', kind='A')

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00164742,현대자동차,005380,Y,반기보고서 (2025.06),20250814003906,현대자동차,20250814,
1,00164742,현대자동차,005380,Y,분기보고서 (2025.03),20250515001159,현대자동차,20250515,
2,00164742,현대자동차,005380,Y,사업보고서 (2024.12),20250312001148,현대자동차,20250312,연
3,00164742,현대자동차,005380,Y,분기보고서 (2024.09),20241114002593,현대자동차,20241114,
4,00164742,현대자동차,005380,Y,반기보고서 (2024.06),20240814004199,현대자동차,20240814,
...,...,...,...,...,...,...,...,...,...
100,00164742,현대자동차,005380,Y,반기보고서 (2000.06),20000814000121,현대자동차,20000814,
101,00164742,현대자동차,005380,Y,분기보고서 (2000.03),20000515000428,현대자동차,20000515,
102,00164742,현대자동차,005380,Y,[기재정정]사업보고서 (1999.12),20000503000073,현대자동차,20000503,연
103,00164742,현대자동차,005380,Y,[기재정정]반기보고서 (1999.06),19991209000001,현대자동차,19991209,


In [17]:
import requests
from bs4 import BeautifulSoup, Tag
import pandas as pd

def url2html(url: str) -> BeautifulSoup | None:
    """
    URL을 입력받아 HTML 내용을 가져온 후 BeautifulSoup 객체로 변환합니다.

    Args:
        url (str): 분석할 웹 페이지의 URL.

    Returns:
        BeautifulSoup | None: 성공 시 파싱된 BeautifulSoup 객체, 실패 시 None.
    """
    try:
        # 지정된 URL에 HTTP GET 요청을 보냄 (타임아웃 10초)
        response = requests.get(url, timeout=10)
        
        # 200 OK 상태 코드가 아닐 경우 예외를 발생시켜 에러 처리
        response.raise_for_status()

        # 응답 받은 HTML 텍스트를 BeautifulSoup 객체로 파싱
        soup = BeautifulSoup(response.text, 'html.parser')
        
        return str(soup)

    except requests.exceptions.RequestException as e:
        # 네트워크 오류, 타임아웃, 잘못된 URL 등의 요청 관련 에러 처리
        print(f"Error fetching URL '{url}': {e}")
        return None
    except Exception as e:
        # 기타 예외 처리
        print(f"An unexpected error occurred: {e}")
        return None
    
    
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    # 비어있으면 그대로 반환
    if df.empty:
        return df

    # 1. 컬럼 문자열화
    df.columns = df.columns.map(str)

    # 2. 멀티헤더 처리
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            ' '.join([str(i) for i in col if 'Unnamed' not in str(i)]).strip()
            for col in df.columns.values
        ]
    
    # 3. 일반 컬럼도 Unnamed 제거
    df.columns = [c.replace('Unnamed: 0', '').strip() for c in df.columns]

    # 4. 첫 컬럼명이 비어있으면 '구분'으로 변경
    if df.columns[0] == '':
        df.rename(columns={df.columns[0]: '구분'}, inplace=True)

    # 5. NaN → 빈 문자열
    df = df.fillna('')

    # 💡 항상 DataFrame을 반환하도록 수정
    return df

In [21]:
url = dart.sub_docs('20250814003156').iloc[20]['url']
html = url2html(url)
dfs = pd.read_html(html)
dfs_markdown = []
for df in dfs:
    if isinstance(df, pd.DataFrame):
        # 1. 데이터프레임을 먼저 정리
        cleaned_df = clean_dataframe(df)

        # 2. 정리된 데이터프레임의 컬럼 수에 따라 분기
        if cleaned_df.shape[1] == 1:
            # 컬럼이 1개면 직접 문자열로 변환
            content = '\n\n'.join([str(item) for sublist in cleaned_df.values.tolist() for item in sublist])
            dfs_markdown.append(content)
        else:
            # 컬럼이 여러 개면 to_markdown 호출
            dfs_markdown.append(cleaned_df.to_markdown(index=False))
    else:
        # DataFrame이 아닌 다른 타입은 그대로 추가
        dfs_markdown.append(df)

# financial_statement 합치는 코드는 동일
financial_statement = ''
for item in dfs_markdown:
    financial_statement += '\n' + str(item) + '\n' # str(item)으로 안정성 추가

/var/folders/kr/rb7ljbw51r1cxfhjn3qz34240000gn/T/ipykernel_71033/1841267722.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)


In [22]:
print(financial_statement)


재무상태표

제 57 기 반기말 2025.06.30 현재

제 56 기말 2024.12.31 현재

(단위 : 백만원)

| 구분                                      | 제 57 기 반기말   | 제 56 기말   |
|:------------------------------------------|:------------------|:-------------|
| 자산                                      |                   |              |
| 유동자산                                  | 73013980          | 82320322     |
| 현금및현금성자산 (주3,26)                 | 6340234           | 1653766      |
| 단기금융상품 (주3,26)                     | 400291            | 10187991     |
| 매출채권 (주3,26)                         | 32533367          | 33840357     |
| 미수금 (주3,26)                           | 1879587           | 3249731      |
| 선급비용                                  | 1491385           | 1381781      |
| 재고자산 (주5)                            | 27102449          | 29154115     |
| 기타유동자산 (주3,26)                     | 3266667           | 2852581      |
| 비유동자산                                | 249003419         | 242645805    |
| 기타포괄손익-공정가치금융자산 (주3

In [27]:
print(str(dart.sub_docs('20250814003156')['title']))


0                       반 기 보 고 서
1                  【 대표이사 등의 확인 】
2                       I. 회사의 개요
3                       1. 회사의 개요
4                       2. 회사의 연혁
5                     3. 자본금 변동사항
6                     4. 주식의 총수 등
7                    5. 정관에 관한 사항
8                      II. 사업의 내용
9                       1. 사업의 개요
10                 2. 주요 제품 및 서비스
11                  3. 원재료 및 생산설비
12                   4. 매출 및 수주상황
13                 5. 위험관리 및 파생거래
14               6. 주요계약 및 연구개발활동
15                     7. 기타 참고사항
16                 III. 재무에 관한 사항
17                      1. 요약재무정보
18                      2. 연결재무제표
19                   3. 연결재무제표 주석
20                        4. 재무제표
21                     5. 재무제표 주석
22                   6. 배당에 관한 사항
23      7. 증권의 발행을 통한 자금조달에 관한 사항
24                8. 기타 재무에 관한 사항
25            IV. 이사의 경영진단 및 분석의견
26               V. 회계감사인의 감사의견 등
27                 1. 외부감사에 관한 사항
28                 2. 내부통제에 관한 사항
29        VI. 

ImportError: attempted relative import with no known parent package